# Technosphere flows induced by the REMIND electricity sector

We add endpoints to the *biosphere* that reflect technosphere flows. This way we can account for material flows in the REMIND inventory.

In [1]:
%run initialize_notebook.ipynb

In [63]:
databases

Databases dictionary with 10 object(s):
	Carma CCS
	Inventory flows
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

In [15]:
eidb = Database("ecoinvent_Remind_BAU_2030")

In [ ]:
available_tech_markets = []
for db in databases:
    if db.startswith("ecoinvent_REMIND_"):
        print("Search tech markets for {}.".format(db))
        
        eidb = Database(db)
        techno_markets = {
            "steel": [act for act in eidb if "market for steel," in act["name"] and
                    "hot rolled" not in act["name"]],
            "concrete": [act for act in eidb if "market for concrete," in act["name"]],
            "copper": [act for act in eidb if "market for copper" == act["name"]],
            "aluminium": [act for act in eidb if "market for aluminium, primary" in act["name"] or
                "market for aluminium alloy," in act["name"]],
            "electricity": [act for act in eidb if act["name"].startswith("market for electricity")],
            "gas": [act for act in eidb if act["name"].startswith("market for natural gas,") and
               not "network" in act["name"] and
               not "burned" in act["name"]],
            "diesel": [act for act in eidb if "market for diesel" in act["name"] and
                not "burned" in act["name"] and
                not "electric" in act["name"]],
            "petrol": [act for act in eidb if "market for petrol," in act["name"] and
                not "burned" in act["name"]],
            "freight": [act for act in eidb if "market for transport, freight" in act["name"]],
            "cement": [act for act in eidb if act["name"].startswith("market for cement,")],
            "heat": [act for act in eidb if act["name"].startswith("market for heat,")]
        }
        
        available_tech_markets = list(techno_markets.keys())
        
        
        print("Check for consistent units across technologies.")
        for kind, actlst in techno_markets.items():
            for act in actlst:
                if act["unit"] != actlst[0]["unit"]:
                    print("Activity `{}` of kind {} has unit {}.".format(act, kind, act["unit"]))
                    raise("Units are not aligned!")
                    
        print("Add inventory flows to database.")
        inventory = Database("Inventory flows")
        for kind in techno_markets:
            if not [act for act in inventory if act["name"] == kind]:
                inventory.new_activity(kind, **{
                    "name": kind,
                    "unit": techno_markets[kind][0]["unit"],
                    "type": "inventory flow",
                    "categories": ("inventory",),
                })
        
        print("Add flows to activities.")
        for kind, actlst in techno_markets.items():
            for act in actlst:
            # clear exsiting exchanges
            # [ex.delete() for ex in act.exchanges() if ex["input"] == ("Inventory flows", kind)]
            if not [ex for ex in act.exchanges() if ex["input"] == ("Inventory flows", kind)]:
                act.new_exchange(**{
                    'input': ('Inventory flows', kind),
                    'type': 'biosphere',
                    'amount': 1
                }).save()

Now add the new inventory (biosphere-like) flows to the activities in question:

In [62]:
databases

Databases dictionary with 10 object(s):
	Carma CCS
	Inventory flows
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050